<a href="https://colab.research.google.com/github/Barleysack/tensor101/blob/Handsonml2_7/Dataset_Too_big_op_at_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf 
from tensorflow import keras
from sklearn.datasets import load_sample_image
import numpy as np
from functools import partial
#데이터셋이 생각보다 용량이 커서 코랩으로 구동 예정.

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
dataset_size = info.splits["train"].num_examples #데이터셋의 데이터 갯수
class_names = info.features["label"].names #클래스 레이블(꽃 이름)
n_classes = info.features["label"].num_classes #클래스 갯수

test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True) #스플릿 API가 책 출판 당시보다 발전. 저렇게만 쓰면 RAW SET은 완성.




def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label #리사이징, XCEPTION 모델의 전처리 함수 사용.


batch_size = 32
train_set = train_set_raw.shuffle(1000)
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [2]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False) #최상층에 해당하는 층들 제외. 
avg = keras.layers.GlobalAveragePooling2D()(base_model.output) #제외한 곳에 전역 평균 풀링 추가.
output = keras.layers.Dense(n_classes, activation="softmax")(avg)#클래스마다 유닛하나, 소프트맥스 덴스층 .
model = keras.models.Model(inputs=base_model.input, outputs=output)# 모델 클래스 객체 선언.

83689472/83683744 [==============================] - 0s 0us/step


In [3]:
for layer in base_model.layers:
  layer.trainable=False#가져온 초기 층들은 가중치 동결.


In [4]:


optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)
#몇번의 반복 후, 검증 정확도가 더이상 변하지 않으면 최상위 층이 모두 훈련된 것.
#이후 모든 층 동결 해제.
#동결/동결 해제시 컴파일 필수.
#가중치 손상 막기 위해 다음엔 학습률 小



Epoch 1/5
86/86 [==============================] - 41s 103ms/step - loss: 1.3064 - accuracy: 0.7268 - val_loss: 1.1434 - val_accuracy: 0.8088
Epoch 2/5
86/86 [==============================] - 8s 90ms/step - loss: 0.3880 - accuracy: 0.9058 - val_loss: 0.8207 - val_accuracy: 0.8493
Epoch 3/5
86/86 [==============================] - 8s 90ms/step - loss: 0.2002 - accuracy: 0.9419 - val_loss: 0.6033 - val_accuracy: 0.8676
Epoch 4/5
86/86 [==============================] - 8s 90ms/step - loss: 0.0772 - accuracy: 0.9722 - val_loss: 0.6022 - val_accuracy: 0.8732
Epoch 5/5
86/86 [==============================] - 8s 90ms/step - loss: 0.0608 - accuracy: 0.9780 - val_loss: 0.6075 - val_accuracy: 0.8713


In [7]:
#for layer in base_model.layers:
layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=40)

Epoch 1/40
86/86 [==============================] - 11s 98ms/step - loss: 0.0413 - accuracy: 0.9893 - val_loss: 0.5913 - val_accuracy: 0.8658
Epoch 2/40
86/86 [==============================] - 8s 90ms/step - loss: 0.0297 - accuracy: 0.9938 - val_loss: 0.5964 - val_accuracy: 0.8713
Epoch 3/40
86/86 [==============================] - 8s 90ms/step - loss: 0.0315 - accuracy: 0.9923 - val_loss: 0.5921 - val_accuracy: 0.8713
Epoch 4/40
86/86 [==============================] - 8s 90ms/step - loss: 0.0311 - accuracy: 0.9950 - val_loss: 0.5901 - val_accuracy: 0.8695
Epoch 5/40
86/86 [==============================] - 8s 90ms/step - loss: 0.0260 - accuracy: 0.9966 - val_loss: 0.5917 - val_accuracy: 0.8732
Epoch 6/40
86/86 [==============================] - 8s 90ms/step - loss: 0.0247 - accuracy: 0.9969 - val_loss: 0.5898 - val_accuracy: 0.8713
Epoch 7/40
86/86 [==============================] - 8s 90ms/step - loss: 0.0263 - accuracy: 0.9942 - val_loss: 0.5893 - val_accuracy: 0.8713
Epoch 8/40
8

In [ ]:
#책처럼 95퍼센트의 정확도는 도달하지 못하였다. 